# Prototype: Data Preparation

In [152]:
import open3d as o3d
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable

import sys
sys.path.insert(0, '../impl/utils/')
import voxel_processing as vp

# device will determine whether to run the training on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
# Create datasets
vp.preprocess_files('../data/obj_test_data/', '../data/voxel_data')

Processing: ../data/obj_test_data/voxel_grid_1.obj -> ../data/voxel_data/voxel_grid_0
Processing: ../data/obj_test_data/voxel_grid_0.obj -> ../data/voxel_data/voxel_grid_1
2 file(s) have been processed.


In [8]:
# Load datasets
data = vp.read_dataset_from_path('../data/voxel_data/')
data = vp.prepare_dataset(data)
data

Reading: ../data/voxel_data/voxel_grid_1.obj
Reading: ../data/voxel_data/voxel_grid_0.omap
Reading: ../data/voxel_data/voxel_grid_1.offsetvec
Reading: ../data/voxel_data/voxel_grid_0.gridpoints
Reading: ../data/voxel_data/voxel_grid_0.obj
Reading: ../data/voxel_data/voxel_grid_1.omap
Reading: ../data/voxel_data/voxel_grid_0.offsetvec
Reading: ../data/voxel_data/voxel_grid_1.gridpoints
2 dataset(s) have been processed. 
2 dataset(s) have been prepared. 


([TriangleMesh with 6130 points and 8448 triangles.,
  TriangleMesh with 115743 points and 231482 triangles.],
 tensor([[[[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],
 
           [[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],
 
           [[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],
 
           ...,
 
           [[0., 0.

In [9]:
mesh, omap, grid_points, offset_vector, sample_points = data[0][0], data[1][0], data[2][0], data[3][0], data[4][0]

pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(np.array(sample_points[:] + offset_vector)))

o3d.visualization.draw_geometries([mesh, pcd])

##### Test `compute_batch_std_grid_indices`

In [10]:
a = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]], dtype=torch.float32)
a

tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.]],

        [[ 7.,  8.,  9.],
         [10., 11., 12.]]])

In [11]:
a[:, 0].shape

torch.Size([2, 3])

In [12]:
primitive_coords = torch.linspace(1, 13, 24)
primitive_coords

tensor([ 1.0000,  1.5217,  2.0435,  2.5652,  3.0870,  3.6087,  4.1304,  4.6522,
         5.1739,  5.6957,  6.2174,  6.7391,  7.2609,  7.7826,  8.3043,  8.8261,
         9.3478,  9.8696, 10.3913, 10.9130, 11.4348, 11.9565, 12.4783, 13.0000])

In [13]:
n_samples = a.shape[1]
tmp0 = a.transpose(1, 2)

xs = tmp0[:, 0].reshape((1, -1))
ys = tmp0[:, 1].reshape((1, -1))
zs = tmp0[:, 2].reshape((1, -1))

x = torch.searchsorted(primitive_coords, xs) - 1
y = torch.searchsorted(primitive_coords, ys) - 1
z = torch.searchsorted(primitive_coords, zs) - 1

x[x < 0] = 0
y[y < 0] = 0
z[z < 0] = 0

x = x.reshape((-1, 1, n_samples))
y = y.reshape((-1, 1, n_samples))
z = z.reshape((-1, 1, n_samples))

torch.concat([x, y, z], dim=1).transpose(1, 2)

/tmp/ipykernel_206291/2877248056.py:8: UserWarning: torch.searchsorted(): input value tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous input value tensor if possible. This message will only appear once per program. (Triggered internally at ../aten/src/ATen/native/BucketizationUtils.h:32.)
  x = torch.searchsorted(primitive_coords, xs) - 1


tensor([[[ 0,  1,  3],
         [ 5,  7,  9]],

        [[11, 13, 15],
         [17, 19, 21]]])

In [14]:
vector_norm = torch.linalg.vector_norm(a, dim=2)
result = torch.sum(vector_norm, dim=1)
print(result.shape)
result

torch.Size([2])


tensor([12.5166, 33.0334])

##### Planar Transformation

In [47]:
plane = torch.tensor([[3, 4, 5, 7], [1, -1, -4, 2]], dtype=torch.float32)
n_norm = torch.norm(plane[:, 0:3], dim=1)
print(n_norm)
tmp0 = plane[:, 0:3]
n = (tmp0.T / n_norm).T
n

tensor([7.0711, 4.2426])


tensor([[ 0.4243,  0.5657,  0.7071],
        [ 0.2357, -0.2357, -0.9428]])

In [48]:
q = torch.rand((2, 5, 3), dtype=torch.float32)
q

tensor([[[0.2558, 0.7013, 0.1370],
         [0.3984, 0.2373, 0.3159],
         [0.2411, 0.0107, 0.0709],
         [0.8064, 0.7516, 0.0249],
         [0.9218, 0.1792, 0.1300]],

        [[0.3738, 0.8851, 0.2730],
         [0.0683, 0.8663, 0.5299],
         [0.8726, 0.5157, 0.1738],
         [0.1531, 0.0543, 0.7369],
         [0.4746, 0.6482, 0.9395]]])

In [50]:
plane[:, 3]

tensor([7., 2.])

In [49]:
coeff = (torch.einsum('bij,bj->bi', q, n).T + plane[:, 3]).T
coeff

tensor([[7.6021, 7.5266, 7.1585, 7.7849, 7.5844],
        [1.6221, 1.3123, 1.9203, 1.3285, 1.0733]])

In [71]:
M, N = 2, 5
# coeff1 = torch.concat([coeff0, coeff0, coeff0], dim=2)
# coeff1

In [77]:
coeff0 = coeff.reshape(M, N)
coeff0

tensor([[7.6021, 7.5266, 7.1585, 7.7849, 7.5844],
        [1.6221, 1.3123, 1.9203, 1.3285, 1.0733]])

In [80]:
coeff1 = coeff0.reshape(1, M * N)
coeff1

tensor([[7.6021, 7.5266, 7.1585, 7.7849, 7.5844, 1.6221, 1.3123, 1.9203, 1.3285,
         1.0733]])

In [65]:
n0 = n.reshape(M, 1, 3).repeat([1, N, 1])
n0

tensor([[[ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071]],

        [[ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428]]])

In [69]:
n1 = n.reshape(M, 1, 3).repeat([1, N, 1]).reshape(1, M * N, 3)
n1

tensor([[[ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428]]])

In [85]:
n0

tensor([[[ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071],
         [ 0.4243,  0.5657,  0.7071]],

        [[ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428],
         [ 0.2357, -0.2357, -0.9428]]])

In [86]:
coeff

tensor([[7.6021, 7.5266, 7.1585, 7.7849, 7.5844],
        [1.6221, 1.3123, 1.9203, 1.3285, 1.0733]])

In [82]:
tmp = n1.transpose(1, 2) * coeff1
tmp

tensor([[[ 3.2253,  3.1933,  3.0371,  3.3028,  3.2178,  0.3823,  0.3093,
           0.4526,  0.3131,  0.2530],
         [ 4.3004,  4.2577,  4.0494,  4.4038,  4.2904, -0.3823, -0.3093,
          -0.4526, -0.3131, -0.2530],
         [ 5.3755,  5.3221,  5.0618,  5.5047,  5.3630, -1.5293, -1.2373,
          -1.8104, -1.2525, -1.0119]]])

In [83]:
tmp0 = tmp.transpose(1, 2).reshape(M, N, 3)
tmp0

tensor([[[ 3.2253,  4.3004,  5.3755],
         [ 3.1933,  4.2577,  5.3221],
         [ 3.0371,  4.0494,  5.0618],
         [ 3.3028,  4.4038,  5.5047],
         [ 3.2178,  4.2904,  5.3630]],

        [[ 0.3823, -0.3823, -1.5293],
         [ 0.3093, -0.3093, -1.2373],
         [ 0.4526, -0.4526, -1.8104],
         [ 0.3131, -0.3131, -1.2525],
         [ 0.2530, -0.2530, -1.0119]]])

In [87]:
# TODO: Compute the q_prime, given batch_n (normalized) and coefficients
q - 2 * tmp0

tensor([[[ -6.1948,  -7.8995, -10.6140],
         [ -5.9881,  -8.2781, -10.3284],
         [ -5.8331,  -8.0882, -10.0527],
         [ -5.7993,  -8.0560, -10.9846],
         [ -5.5137,  -8.4016, -10.5959]],

        [[ -0.3909,   1.6498,   3.3316],
         [ -0.5503,   1.4850,   3.0044],
         [ -0.0327,   1.4209,   3.7947],
         [ -0.4732,   0.6805,   3.2420],
         [ -0.0314,   1.1541,   2.9634]]])

##### Batch Planar Transform based on Batch Planes (M, 3, 4)

In [92]:
planes = torch.tensor([[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],
                       [[-1, -2, -3, -4], [-5, -6, -7, -8], [-9, -10, -11, -12]]], dtype=torch.float32)
planes

tensor([[[  1.,   2.,   3.,   4.],
         [  5.,   6.,   7.,   8.],
         [  9.,  10.,  11.,  12.]],

        [[ -1.,  -2.,  -3.,  -4.],
         [ -5.,  -6.,  -7.,  -8.],
         [ -9., -10., -11., -12.]]])

In [93]:
points = torch.tensor([[[1, 3, 4], [6, 7, 2]], [[7, 3, 3], [5, 2, 1]]], dtype=torch.float32)
points

tensor([[[1., 3., 4.],
         [6., 7., 2.]],

        [[7., 3., 3.],
         [5., 2., 1.]]])

In [149]:
n_raw = torch.tensor([5, 6, 7], dtype=torch.float32)
n1 = n_raw / torch.norm(n_raw)
torch.tensor([1, 3, 4.]) - 25.7253 * n1

tensor([-11.2641, -11.7169, -13.1697])

In [146]:
def apply_planar_transform(batch_sample_points, batch_plane):
    '''
    Compute, for each sample, sample points after planar reflective transformation.
    
    The formula is given by `q' = q - 2 <q - r, n> n`, where:
    - q is the target point, q' is the point after symmetric transformation,
    - r is the orthogonal displacement vector of the plane, and
    - n is the normalized normal vector of the plane.
    '''
    M = batch_sample_points.shape[0]
    N = batch_sample_points.shape[1]
    D = batch_plane.shape[1]
    
    q = batch_sample_points
    
    n_norm = torch.norm(batch_plane[:, :, 0:3], dim=2).reshape(M, D, 1)
    print(n_norm)
    print('\nnormals:')
    print(batch_plane[:, :, 0:3])
    n = (batch_plane[:, :, 0:3] / n_norm).transpose(1, 2)
    print('\nn:')
    print(n)
    # print('\nEinstein sum:')
    # print(torch.einsum('bij,bjk->bik', q, n))
    d = batch_plane[:, :, 3].reshape(M, 1, D).repeat([1, N, 1])
    print('\nd:')
    print(d)
    coeff = 2. * (torch.einsum('bij,bjk->bik', q, n) + d)
    print('\ncoeff:')
    print(coeff)
    
    coeff1 = coeff.reshape(M, N, D, 1).repeat([1, 1, 1, 3])
    print('\ncoeff1:')
    print(coeff1)
    n0 = n.transpose(1, 2).reshape(M, 1, D, 3).repeat([1, N, 1, 1])
    print('\nn0:')
    print(n0)
    tmp = coeff1 * n0
    print('\ndir_vec:')
    print(tmp)
    
    q0 = q.reshape(M, N, 1, 3).repeat([1, 1, D, 1])
    
    result = q0 - tmp
    print('\nresult:')
    print(result)
    
    return result

apply_planar_transform(points, planes)

tensor([[[ 3.7417],
         [10.4881],
         [17.3781]],

        [[ 3.7417],
         [10.4881],
         [17.3781]]])

normals:
tensor([[[  1.,   2.,   3.],
         [  5.,   6.,   7.],
         [  9.,  10.,  11.]],

        [[ -1.,  -2.,  -3.],
         [ -5.,  -6.,  -7.],
         [ -9., -10., -11.]]])

n:
tensor([[[ 0.2673,  0.4767,  0.5179],
         [ 0.5345,  0.5721,  0.5754],
         [ 0.8018,  0.6674,  0.6330]],

        [[-0.2673, -0.4767, -0.5179],
         [-0.5345, -0.5721, -0.5754],
         [-0.8018, -0.6674, -0.6330]]])

d:
tensor([[[  4.,   8.,  12.],
         [  4.,   8.,  12.]],

        [[ -4.,  -8., -12.],
         [ -4.,  -8., -12.]]])

coeff:
tensor([[[ 18.1559,  25.7253,  33.5522],
         [ 21.8976,  32.3996,  40.8027]],

        [[-19.7595, -30.1112, -38.5010],
         [-14.4143, -24.3905, -32.7466]]])

coeff1:
tensor([[[[ 18.1559,  18.1559,  18.1559],
          [ 25.7253,  25.7253,  25.7253],
          [ 33.5522,  33.5522,  33.5522]],

         [[ 21.

tensor([[[[ -3.8524,  -6.7048, -10.5571],
          [-11.2641, -11.7169, -13.1697],
          [-16.3764, -16.3071, -17.2379]],

         [[  0.1476,  -4.7048, -15.5571],
          [ -9.4459, -11.5351, -19.6242],
          [-15.1314, -16.4793, -23.8273]]],


        [[[  1.7191,  -7.5619, -12.8428],
          [ -7.3550, -14.2260, -17.0970],
          [-12.9393, -19.1548, -21.3703]],

         [[  1.1476,  -5.7048, -10.5571],
          [ -6.6277, -11.9532, -15.2788],
          [-11.9592, -16.8436, -19.7279]]]])

##### Axial Transform

In [175]:
quats_id = torch.tensor([[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]],
                         [[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]], dtype=torch.float32)
quats_id

tensor([[[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]],

        [[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]]])

In [296]:
quats = torch.tensor([[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]],
                      [[-1, -2, -3, -4], [-5, -6, -7, -8], [-9, -10, -11, -12]]], dtype=torch.float32)
quats

tensor([[[  1.,   2.,   3.,   4.],
         [  5.,   6.,   7.,   8.],
         [  9.,  10.,  11.,  12.]],

        [[ -1.,  -2.,  -3.,  -4.],
         [ -5.,  -6.,  -7.,  -8.],
         [ -9., -10., -11., -12.]]])

In [297]:
points = torch.tensor([[[1, 3, 4], [6, 7, 2]], [[7, 3, 3], [5, 2, 1]]], dtype=torch.float32)
points

tensor([[[1., 3., 4.],
         [6., 7., 2.]],

        [[7., 3., 3.],
         [5., 2., 1.]]])

In [291]:
def multiply_quaternion(r, s):
    result = torch.zeros_like(r)
    
    result[0] = r[0] * s[0] - r[1] * s[1] - r[2] * s[2] - r[3] * s[3]
    result[1] = r[0] * s[1] + r[1] * s[0] + r[2] * s[3] - r[3] * s[2]
    result[2] = r[0] * s[2] - r[1] * s[3] + r[2] * s[0] + r[3] * s[1]
    result[3] = r[0] * s[3] + r[1] * s[2] - r[2] * s[1] + r[3] * s[0]
    
    return result

p = quats[0, 0].clone().detach()
p[0] = 0.
print(p)
q_proto = quats[0, 2]
# q_proto = torch.tensor([1., 0, 0, 0])
q = q_proto / torch.norm(q_proto)
print(f"q = {q}")
print(f"q_norm = {torch.norm(q)}")
qp = q.clone().detach()
qp[1:4] = -qp[1:4]
print(f"qp = {qp}")
print(f"qp_norm = {torch.norm(qp)}")

t1 = multiply_quaternion(p, q)
print(f"p*q = {t1}")
multiply_quaternion(qp, t1)

tensor([0., 2., 3., 4.])
q = tensor([0.4262, 0.4735, 0.5209, 0.5682])
q_norm = 1.0
qp = tensor([ 0.4262, -0.4735, -0.5209, -0.5682])
qp_norm = 1.0
p*q = tensor([-4.7825,  0.4735,  2.0361,  1.3258])


tensor([-1.7881e-07,  2.9327e+00,  3.7175e+00,  2.5650e+00])

In [250]:
quats

tensor([[[  0.,   2.,   3.,   4.],
         [  5.,  -6.,  -7.,  -8.],
         [  9., -10., -11., -12.]],

        [[ -1.,   2.,   3.,   4.],
         [ -5.,   6.,   7.,   8.],
         [ -9.,  10.,  11.,  12.]]])

In [161]:
quats0 = quats
quats0[:, :, 1:4] = -quats0[:, :, 1:4]
quats0

tensor([[[  1.,  -2.,  -3.,  -4.],
         [  5.,  -6.,  -7.,  -8.],
         [  9., -10., -11., -12.]],

        [[ -1.,   2.,   3.,   4.],
         [ -5.,   6.,   7.,   8.],
         [ -9.,  10.,  11.,  12.]]])

In [298]:
def batch_multiply_quaternion(r, s):
    M = r.shape[0]
    N = r.shape[1]
    D = r.shape[2]
    print(f"M, N, D = {M, N, D}")
    
    result_r = torch.zeros((M, N, D), requires_grad=True)
    result_i = torch.zeros((M, N, D), requires_grad=True)
    result_j = torch.zeros((M, N, D), requires_grad=True)
    result_k = torch.zeros((M, N, D), requires_grad=True)
    result_r = r[:, :, :, 0] * s[:, :, :, 0] - r[:, :, :, 1] * s[:, :, :, 1] \
                - r[:, :, :, 2] * s[:, :, :, 2] - r[:, :, :, 3] * s[:, :, :, 3]
    result_i = r[:, :, :, 1] * s[:, :, :, 0] + r[:, :, :, 0] * s[:, :, :, 1] \
                + r[:, :, :, 2] * s[:, :, :, 3] - r[:, :, :, 3] * s[:, :, :, 2]
    result_j = r[:, :, :, 2] * s[:, :, :, 0] + r[:, :, :, 0] * s[:, :, :, 2] \
                + r[:, :, :, 3] * s[:, :, :, 1] - r[:, :, :, 1] * s[:, :, :, 3]
    result_k = r[:, :, :, 3] * s[:, :, :, 0] + r[:, :, :, 0] * s[:, :, :, 3] \
                + r[:, :, :, 1] * s[:, :, :, 2] - r[:, :, :, 2] * s[:, :, :, 1]
    print(result_r.shape)
                
    result_r = result_r.reshape(M, N, D, 1)
    result_i = result_i.reshape(M, N, D, 1)
    result_j = result_j.reshape(M, N, D, 1)
    result_k = result_k.reshape(M, N, D, 1)
    return torch.cat([result_r, result_i, result_j, result_k], dim=3)

batch_quats = quats.reshape(2, 1, 3, 4).repeat([1, 2, 1, 1])
batch_multiply_quaternion(batch_quats, batch_quats)

M, N, D = (2, 2, 3)
torch.Size([2, 2, 3])


tensor([[[[ -28.,    4.,    6.,    8.],
          [-124.,   60.,   70.,   80.],
          [-284.,  180.,  198.,  216.]],

         [[ -28.,    4.,    6.,    8.],
          [-124.,   60.,   70.,   80.],
          [-284.,  180.,  198.,  216.]]],


        [[[ -28.,    4.,    6.,    8.],
          [-124.,   60.,   70.,   80.],
          [-284.,  180.,  198.,  216.]],

         [[ -28.,    4.,    6.,    8.],
          [-124.,   60.,   70.,   80.],
          [-284.,  180.,  198.,  216.]]]])

In [267]:
def batch_normalize(batch_quaternions):
    M = batch_quaternions.shape[0]
    D = batch_quaternions.shape[1]
    
    q_norm = torch.norm(batch_quaternions, dim=2).reshape(M, D, 1)
    q = batch_quaternions / q_norm
    return q

In [299]:
def apply_quaternion_rotation(batch_sample_points, batch_quaternions):
    '''
    Compute, for each sample, sample points after rotation using quaternions.
    '''
    M = batch_sample_points.shape[0]
    N = batch_sample_points.shape[1]
    D = batch_quaternions.shape[1]
    
    p = torch.cat([torch.zeros((M, N, 1)), batch_sample_points], dim=2).reshape(M, N, 1, 4).repeat([1, 1, D, 1])
    print(f'\np:')
    print(p)
    # normalized the quaternion first
    q = batch_normalize(batch_quaternions)
    # prepare for quaternion multiplication
    q0 = q.reshape(M, 1, D, 4).repeat([1, N, 1, 1])
    print(f'\nq0:')
    print(q0)
    q0p = q0.clone()
    q0p_im = -q0[:, :, :, 1:4].clone()
    q0p_re = q0[:, :, :, 0].clone().reshape(M, N, D, 1)
    q0p = torch.cat([q0p_re, q0p_im], dim=3)
    print(f'\nq0p:')
    print(q0p)
    
    tmp0 = batch_multiply_quaternion(p, q0)
    tmp0 = batch_multiply_quaternion(q0p, tmp0)
    
    return tmp0[:, :, :, 0:3].clone()

apply_quaternion_rotation(points, quats)


p:
tensor([[[[0., 1., 3., 4.],
          [0., 1., 3., 4.],
          [0., 1., 3., 4.]],

         [[0., 6., 7., 2.],
          [0., 6., 7., 2.],
          [0., 6., 7., 2.]]],


        [[[0., 7., 3., 3.],
          [0., 7., 3., 3.],
          [0., 7., 3., 3.]],

         [[0., 5., 2., 1.],
          [0., 5., 2., 1.],
          [0., 5., 2., 1.]]]])

q0:
tensor([[[[ 0.1826,  0.3651,  0.5477,  0.7303],
          [ 0.3790,  0.4549,  0.5307,  0.6065],
          [ 0.4262,  0.4735,  0.5209,  0.5682]],

         [[ 0.1826,  0.3651,  0.5477,  0.7303],
          [ 0.3790,  0.4549,  0.5307,  0.6065],
          [ 0.4262,  0.4735,  0.5209,  0.5682]]],


        [[[-0.1826, -0.3651, -0.5477, -0.7303],
          [-0.3790, -0.4549, -0.5307, -0.6065],
          [-0.4262, -0.4735, -0.5209, -0.5682]],

         [[-0.1826, -0.3651, -0.5477, -0.7303],
          [-0.3790, -0.4549, -0.5307, -0.6065],
          [-0.4262, -0.4735, -0.5209, -0.5682]]]])

q0p:
tensor([[[[ 0.1826, -0.3651, -0.5477, -0.7303],
   

tensor([[[[ 1.4901e-08,  2.6667e+00,  2.8667e+00],
          [ 1.1921e-07,  3.1264e+00,  3.5287e+00],
          [-3.5763e-07,  3.1211e+00,  3.7085e+00]],

         [[-1.7881e-07,  1.3333e+00,  3.3333e-01],
          [-3.5763e-07,  5.1034e+00,  1.0690e+00],
          [ 1.4901e-07,  5.9013e+00,  1.3856e+00]]],


        [[[ 0.0000e+00, -1.6667e+00,  2.7333e+00],
          [-2.3842e-07,  1.1839e+00,  2.6782e+00],
          [ 3.5763e-07,  1.8969e+00,  2.7668e+00]],

         [[ 1.1921e-07, -1.6667e+00,  9.3333e-01],
          [ 0.0000e+00,  5.4023e-01,  8.0460e-01],
          [-1.1921e-07,  1.1076e+00,  8.5202e-01]]]])